In [ ]:
from sklearn.metrics import jaccard_score
import community as community_louvain
from collections import defaultdict
#import matplotlib.pyplot as plt
from functools import partial
from glob import glob
import igraph as ig
import pandas as pd
import leidenalg
import os

In [ ]:
# Make dirs for plots
for algorithm in ['leiden', 'louvain', 'multilevel', 'fastgreedy', 'reference']:
    if not os.path.exists(algorithm):
        os.makedirs(algorithm)

In [ ]:
metrics = defaultdict(list)
networks = []

for path in glob("../real/*.net"):
    reference_path = path.replace(".net", ".clu")
    if os.path.exists(reference_path):
        with open(reference_path, 'r') as f:
            next(f)
            reference = [int(line)-1 for line in f]
    else:
        continue
    
    name = os.path.basename(path)[:-4]
    networks.append(name)
    
    g = ig.load(path).simplify()
    reference = ig.VertexClustering(g, reference)
    plot = partial(ig.plot, layout=g.layout_auto())    
    #plot = partial(ig.plot, layout=g.layout_auto(), bbox=(300, 300), margin=20, mark_groups=True)
    
    communities = {
        'leiden': leidenalg.find_partition(g, leidenalg.ModularityVertexPartition),
        #'leiden': g.community_leiden('modularity', n_iterations=-1),        
        'louvain': ig.VertexClustering(
            g,
            community_louvain.best_partition(
                g.to_networkx()
            ).values()
        ),
        'multilevel': g.community_multilevel(),
        'fastgreedy': g.community_fastgreedy().as_clustering(),
        'reference': reference
    }
    
    metric_methods = {
        'vi': partial(ig.compare_communities, reference.membership, method='vi'),
        'nmi': partial(ig.compare_communities, reference.membership, method='nmi'),
        'jaccard': partial(jaccard_score, reference.membership, average='weighted'),
    }
    
    #fig, axs = plt.subplots(2, 2)
    #for ax, (community_name, community) in zip(axs.flat, communities.items()):
    #    plot(community, target=ax)
    #plt.show()
    
    vcount = g.vcount()
    for community_name, community in communities.items():
        with open(f"{community_name}/{name}.clu", "w") as f:
            f.write(f'*Vertices {vcount}\n')
            f.write('\n'.join([str(i+1) for i in community.membership]))
        metrics[community_name + '_mod'].append(community.modularity)
        plot(community, f"{community_name}/{name}.png")
        if community_name == 'reference':
            break
        for metric_name, metric in metric_methods.items():
            metrics[community_name + '_' + metric_name].append(metric(community.membership))
#SAVE TO TABLE
df = pd.DataFrame(metrics, index=networks)
#print("\nMODULARITY TABLE\n", df.filter(regex=".*_mod"))
#print("\nVI TABLE\n", df.filter(regex=".*_vi"))
#print("\nNMI TABLE\n", df.filter(regex=".*_nmi"))
#print("\nJACCARD TABLE\n", df.filter(regex=".*_jaccard"))